In [ ]:
# import numpy as np
import glob
import pypianoroll as ppr
import time
import music21
import os
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from utils.utilsPreprocessing import *
#np.set_printoptions(threshold=np.inf)
#torch.set_printoptions(threshold=50000)

In [ ]:
##########################################
##########HYPERPARAMS#####################
epochs = 5
learning_rate = 1e-1
batch_size = 20
seq_length = 4
log_interval = 1 #Log/show loss per batch
##########################################
##########################################

# Load MIDI files from npz

In [ ]:
data = np.load('../YamahaPianoCompetition2002NoTranspose.npz')

midiDatasetTrain = data['train']
midiDatasetTest = data['test']

data.close()

"""
print("Training set: ({}, {}, {}, {})".format(midiDatasetTrain.size()[0],
                                                midiDatasetTrain.size()[1],
                                                midiDatasetTrain.size()[2],
                                                midiDatasetTrain.size()[3]))
print("Test set: ({}, {}, {}, {})".format(midiDatasetTest.size()[0],
                                                midiDatasetTest.size()[1],
                                                midiDatasetTest.size()[2],
                                                midiDatasetTest.size()[3]))
"""

print("Training set: {}".format(midiDatasetTrain.shape))
print("Test set: {}".format(midiDatasetTest.shape))

In [ ]:
fullPitch = 128
_, _, length, reducedPitch = midiDatasetTrain.shape

In [ ]:
#MODEL FOR TRANSFER LEARNING
from utils.VAE import VAE
from loadModel import loadModel
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


###VARIATIONAL CONV AUTOENCODER############
autoencoderModel = VAE()
pathToModel = '../models/YamahaPC2002_VAE_Reconstruct_NoTW_20Epochs.model'
###########################################

autoencoderModel = loadModel(autoencoderModel, pathToModel, dataParallelModel=False)

# CDVAE

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
midiDatasetTrain = torch.from_numpy(midiDatasetTrain)
trainLoader = torch.utils.data.DataLoader(midiDatasetTrain, batch_size=batch_size, shuffle=False, drop_last=True)

midiDatasetTest = torch.from_numpy(midiDatasetTest)
testLoader = torch.utils.data.DataLoader(midiDatasetTest, batch_size=batch_size, shuffle=False, drop_last=True)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, batch_size=7, lstm_layers=3, hidden_size=400, seq_length=7):
        super(LSTM, self).__init__()
        
        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.lstm_layers = lstm_layers
        self.seq_length = seq_length
        
        ###LSTM###########
        self.lstm = nn.LSTM(input_size=100, hidden_size=self.hidden_size,
                            num_layers=self.lstm_layers, batch_first=True, dropout=0)
        ##################
        ###GRU############
        self.gru = nn.GRU(input_size=100, hidden_size=self.hidden_size, 
                          num_layers=self.lstm_layers, batch_first=True)
        ###LSTMCells######
        self.lstmC1 = nn.LSTMCell(input_size=100, hidden_size=self.hidden_size, bias=True)
        self.drop1 = nn.Dropout(p=0.2)
        self.lstmC2 = nn.LSTMCell(input_size=400, hidden_size=self.hidden_size, bias=True)
        self.drop2 = nn.Dropout(p=0.2)
        self.lstmC3 = nn.LSTMCell(input_size=400, hidden_size=self.hidden_size, bias=True)
        #################
        
        self.fc = nn.Linear(self.hidden_size,100)
        self.reluFC = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.leakyReLU = nn.LeakyReLU(0.1)
        self.tanh = nn.Tanh()
    
    def splitEmbedding(self, embed):     
        if(embed.size()[0]>self.seq_length):
            embedTemp = torch.chunk(embed, int(self.batch_size/self.seq_length),dim=0)
            embed7s = embedTemp[0].unsqueeze(0)
            for emb in embedTemp[1:]:
                #print("inloop");print(emb.unsqueeze(1).size())
                embed7s = torch.cat((embed7s, emb.unsqueeze(0)),dim=0)
                #print("afterconcat");print(embed7s.size())
        else:
            embed7s = embed.unsqueeze(1)
        return embed7s
    
    def hiddenInitLSTM(self,embed):
        hiddenState = torch.randn(self.lstm_layers,embed.size()[0], self.hidden_size).to(device)
        cellState = torch.randn(self.lstm_layers,embed.size()[0], self.hidden_size).to(device)
        return hiddenState,cellState
    
    def hiddenInitGRU(self,embed):
        hiddenState = torch.zeros(self.lstm_layers,embed.size()[0], self.hidden_size).to(device)    
        return hiddenState
    
    
    def hiddenInitLSTMCell(self,embed):
        hiddenState = torch.zeros(int(embed.size()[0]/7),self.hidden_size).to(device)
        cellState = torch.zeros(int(embed.size()[0]/7),self.hidden_size).to(device)
        hS2 = torch.zeros(int(embed.size()[0]/7),self.hidden_size).to(device)
        cS2 = torch.zeros(int(embed.size()[0]/7),self.hidden_size).to(device)
        
        return (hiddenState, cellState), (hS2, cS2)

    def forward(self, embed):#, lenghts):

        h_t, c_t = self.hiddenInitLSTM(embed)
        #h_t = self.hiddenInitGRU(embed)
        
        #embed = nn.utils.rnn.pack_padded_sequence(embed, 7, batch_first=True)
        #(h_t, c_t),(h2_t, c2_t) = self.hiddenInitLSTMCell(embed)
        #print(h_t.size())
        ###LSTMCells######
        #print(embed.size())
        """
        output = []
        for i in range(7):
            h_t, c_t = self.lstmC1(embed[i],(h_t,c_t))
            #print(h_t.size())
            #h_t = self.drop1(h_t)
            #h2_t, c2_t = self.lstmC2(h_t,(h2_t,c2_t))
            
            output.append(h_t)
        #h_t = nn.Dropout(h_t)
        #h_t, c_t = self.lstmC2(h_t)
        lstmOut = torch.Tensor(7,100).to(device)
        lstmOut = torch.cat(output)
        
        #print("lstmout");print(lstmOut.size())
        """
        ###LSTM###########
        lstmOut, (h_t, c_t) = self.lstm(embed,(h_t, c_t))
        #lstmOut, h_t = self.gru(embed, h_t)
        ##################
        #lstmOut = self.reluFC(lstmOut)
        #lstmOut = self.sigmoid(lstmOut)
        #lstmOut = self.tanh(lstmOut)
        #lstmOut = self.leakyReLU(lstmOut)
        
        lstmOut = self.fc(lstmOut)
        
        if(self.training):
            print('embedding');print(embed.size());print(embed[0,2,:20])
            print('lstmOut');print(lstmOut.size());print(lstmOut[0,1,:20])
        
        print('compare lstm ht');print(lstmOut.size());print(h_t.size())
        
        if(embed.size()[0]==7):
            embed = embed.squeeze(1)
        return embed, lstmOut

    

model = LSTM(batch_size=batch_size, seq_length=seq_length).to(device)

#optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-1)
optimizer = optim.RMSprop(model.parameters(),lr=learning_rate, weight_decay=0.9)

def loss_function(embed, lstmOut):
    
    batch_sizeMin1 = embed.size()[0]-1
    MSELoss = nn.MSELoss(reduction='sum')
    BCELoss = nn.BCEWithLogitsLoss(reduction='sum')
    L1Loss = nn.L1Loss(reduction='sum')
    cosLoss = nn.CosineSimilarity(dim=1)
    #BATCHSIZE 7
    if(embed.size()[0]==7):
        print("loss");print(embed.size());print(lstmOut.size())
        lstmOut = torch.squeeze(lstmOut,1)
        mseLSTM = MSELoss(lstmOut[:-1,:],embed[1:,:])
        totalLoss =  mseLSTM / (embed.size()[1]*batch_sizeMin1)

    #BATCHSIZE > 7
    else:
        #print("LOSS");print(embed.size());print(lstmOut.size())
        
        mseLSTM = MSELoss(lstmOut[:,1:,:],embed[:,:-1,:])
        totalLoss = mseLSTM / (embed.size()[0]*(embed.size()[1]-1)*embed.size()[2])
        
        #L1LSTM = L1Loss(lstmOut[:,:-1,:],embed[:,1:,:])
        #totalLoss = L1LSTM / (embed.size()[0]*(embed.size()[1]-1)*embed.size()[2])
        
        #cosSim = 0
        #for lOut, emb in zip(lstmOut, embed):
            #print(lOut[:-1,:].size());print(emb[1:,:].size())
            #cosSim += torch.sum(cosLoss(lOut[:-1,:],emb[1:,:]))
        #print(cosSim)
        #totalLoss = (embed.size()[0]*(embed.size()[1]-1)) - cosSim
        
        #BCELSTM = 0
        #for lOut, emb in zip(lstmOut,embed):
        #    BCELSTM += BCELoss(lOut[:-1,:],emb[1:,:])
        #totalLoss = BCELSTM / (embed.size()[0]*(embed.size()[1]-1))
        
        #CELoss = lstmLoss2(lstmOut[:,:-1,:],embed[:,1:,:])
        
        
    return totalLoss
        

def train(epoch):
    model.train()
    trainLoss = 0

    for batch_idx, data in enumerate(trainLoader):
        #print(batch_idx)
        data = data.float().to(device)
        optimizer.zero_grad()
        embedding, logvar = autoencoderModel.encoder(data)
        #print(embedding.size())
        embedding = torch.rand_like(embedding)
        embedding = torch.abs(embedding)
        embedding = model.splitEmbedding(embedding)
        #print(embedding.size())
        embedding, lstmOut = model(embedding)
        loss = loss_function(embedding, lstmOut)
        
        ###LSTM###############
        #reconPrediction = autoencoderModel.decoder(lstmOut[-1,:,:])
        ######################
        ###LSTMCell###########
        #reconPrediction = autoencoderModel.decoder(lstmOut)
        ######################

        loss.backward()
        trainLoss += loss.item()
        optimizer.step()
        if(batch_idx % log_interval == 0):
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(trainLoader.dataset),
                100. * batch_idx / len(trainLoader),
                loss.item() / (len(data)-(model.batch_size/7))))
        #if(batch_idx==1):
        #   break
    print('====> Epoch: {} Average Loss: {:.4f}'.format(
          epoch, trainLoss / (len(trainLoader.dataset)-batch_idx*(model.batch_size/7))))

def test(epoch):
    model.eval()
    testLoss = 0
    with torch.no_grad():
        for i, data in enumerate(testLoader):
            data = data.float().to(device)
            embedding, logvar = autoencoderModel.encoder(data)
            embedding = model.splitEmbedding(embedding)
            embedding, lstmOut = model(embedding)
            loss = loss_function(embedding, lstmOut)
            ###LSTM#########
            #reconPrediction = autoencoderModel.decoder(lstmOut[:,-1,:])
            ################
            ###LSTMCell#####
            #reconPrediction = autoencoderModel.decoder(lstmOut)
            ################
            testLoss += loss_function(embedding, lstmOut).item()
            
            #if(i==1):
            #    break
    testLoss /= (len(testLoader.dataset)-i*(model.batch_size/7))

    print('====> Test set Loss: {:.4f}'.format(testLoss))

In [ ]:
"""
#LOAD MODEL
pathToModel = '../models/WikifoniaNoTranpose_10Epochs_LSTM_noTW_dropout50.model'

try:
    #LOAD TRAINED MODEL INTO GPU
    if(torch.cuda.is_available()):
        model = torch.load(pathToModel)
        
    #LOAD MODEL TRAINED ON GPU INTO CPU
    else:
        model = torch.load(pathToModel, map_location=lambda storage, loc: storage)
    print("\n--------model restored--------\n")
except:
    print("\n--------no saved model found--------\n")
"""
print('')

In [ ]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

In [ ]:
#torch.save(model,'/media/EXTHD/niciData/models/YamahaPianoComp2002_5Epochs_LSTM_noTW.model')

In [ ]:
np.set_printoptions(precision=2, suppress=True, threshold=1)


In [ ]:
playSeq = 0
pathToSampleSeq = "../DougMcKenzieFiles/train/AHouseis.mid"
if(model.train()):
    model.eval()
if(autoencoderModel.train()):
    autoencoderModel.eval()

###PREDICT 8th SEQUENCE
with torch.no_grad():
    
    sampleNp1 = getSlicedPianorollMatrixNp(pathToSampleSeq)
    sampleNp1 = deleteZeroMatrices(sampleNp1)
    sample = np.expand_dims(sampleNp1[0,:,36:-32],axis=0)
    print(sample.shape)
    for i, sampleNp in enumerate(sampleNp1[playSeq:playSeq+(seq_length-1)]):
        #print(sampleNp.shape)
        if(np.any(sampleNp)):
            sampleNp = sampleNp[:,36:-32]
            sampleNp = np.expand_dims(sampleNp,axis=0)
            sample = np.concatenate((sample,sampleNp),axis=0)
    samplePlay = sample[0,:,:]
    for s in sample[1:]:
        samplePlay = np.concatenate((samplePlay,s),axis=0)
    samplePlay = addCuttedOctaves(samplePlay)
    print(samplePlay.shape)
    #####PREPARE SAMPLE
    sample = torch.from_numpy(sample).float().to(device)
    sample = torch.unsqueeze(sample,1)
    print(sample.size())
    
    #####MODEL##############
    embed,logvar = autoencoderModel.encoder(sample)
    print(embed.size())
    embed = model.splitEmbedding(embed)
    embed, lstmOut = model(embed)
    print(lstmOut[:,:,:].squeeze(1).size())
    #pred = autoencoderModel.decoder(lstmOut[0,:,:])
    pred = autoencoderModel.decoder(lstmOut[:,:,:].squeeze(1))
    ########################
    
    prediction = pred.squeeze(0).squeeze(0).cpu().numpy()
    print(prediction.shape)
    predict = np.squeeze(prediction, axis=1)
    print(predict.shape)
    
    #####PLAY WHOLE LSTM OUT#############
    prediction = predict[0,:,:]
    #print(prediction.shape)
    for pred in predict[1:]:
        prediction = np.concatenate((prediction, pred), axis =0)
    #####################################
    #####PLAY NEXT SEQUENCE##############
    #prediction = predict[-1,:,:]
    #print(prediction.shape)
    #####################################
    #NORMALIZE PREDICTIONS
    #reconstruction /= np.abs(np.max(reconstruction))
    prediction /= np.abs(np.max(prediction))
    #print(prediction)

    #CHECK MIDI ACTIVATIONS IN PREDICTION TO INCLUDE RESTS
    #reconstruction[reconstruction < 0.3] = 0
    prediction[prediction < 0.5] = 0
    print(prediction)


    ###MONOPHONIC OUTPUT MATRIX POLOYPHONIC POSSIBLE WITH ACTIVATION THRESHOLD###
    #score = music21.converter.parse(
    #'WikifoniaServer/samples/The-Doors---Don\'t-you-love-her-Madly?.mid')
    #score.show()

    samplePlay = debinarizeMidi(samplePlay, prediction=False)
    samplePlay = addCuttedOctaves(samplePlay)
    #reconstruction = debinarizeMidi(reconstruction, prediction=True)
    #reconstruction = addCuttedOctaves(reconstruction)
    prediction = debinarizeMidi(prediction, prediction=True)
    prediction = addCuttedOctaves(prediction)
    print("INPUT")
    print(samplePlay.shape)
    pianorollMatrixToTempMidi(samplePlay, show=True,showPlayer=True,autoplay=False)
    #print("RECONSTRUCTION")
    #pianorollMatrixToTempMidi(reconstruction, show=True,
    #                            showPlayer=True,autoplay=True, prediction=True)
    print("PREDICTION")
    pianorollMatrixToTempMidi(prediction, prediction=True, 
                              show=True,showPlayer=True,autoplay=True)        
    print("\n\n")
            

print('')